In [2]:
import os 

# Please specify the following path to WaterMAI test or train set
path = "your_path_to/WaterMAI_dataset/test"
os.chdir(path)

In [3]:
import numpy as np
import shutil
import cv2
import tifffile as tiff

In [4]:
# NDWI
def NDWI(G, NIR):
    """Compute the NDWI
    INPUT : G (np.array) -> the Green band images as a numpy array of float
            NIR (np.array) -> the Near Infrared images as a numpy array of float
    OUTPUT : NDWI (np.array) -> the NDWI
    """
    NDWI = (G - NIR) / (G + NIR + 1e-12)
    return NDWI

def GreenNirNDWI(greennir):
    """ Input
            greennir _ contains only Green and NIR bands 
    """
    # compute NDWI (G, NIR)
    ndwi = NDWI(greennir[:, :, 0], greennir[:, :, 1])
    ndwi = np.round(ndwi).astype(np.uint8)

    # define images to further process : R, NIR, NDWI
    img_to_process = [greennir[:, :, 0], greennir[:, :, 1], ndwi]

    return np.stack(img_to_process, axis=2)

## Generate 'Green + NIR + NDWI' (3 bands) images & masks
- This cell generate 'Green + NIR + NDWI' (3 bands) images from the **images/color** and **images/ir** folders & saves them in **images/gNirNdwi** folder
- This cell generates a label mask named **labels/mask_gnirndwi** for 'Green + NIR + NDWI' images. This label mask is similar to the **labels/mask_co** label mask, with the only difference being the name of each respective folder

In [6]:
# (Green, NIR, NDWI)
rgb_dir = './images/color'
nir_dir = './images/ir'

GNirNdwi_dir = './images/gNirNdwi'
if not os.path.exists(GNirNdwi_dir):
    os.makedirs(GNirNdwi_dir)

# IMAGES
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    gnirndwi_path = os.path.join(GNirNdwi_dir, name + '_gnirndwi.jpg')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    nir_img = cv2.imread(nir_path)
    
    # Construct combination (G, NIR, NDWI)
    green = rgb_img[:, :, 1]
    nir = nir_img[:, :, 0]
    gnirndwi_img = GreenNirNDWI(np.stack([green, nir], axis=2))
    
    # Save the image as a .jpg file
    cv2.imwrite(gnirndwi_path, gnirndwi_img)
    
    
# LABELS
co_label = './labels/mask_co'
gnirndwi_label = './labels/mask_gnirndwi'
if not os.path.exists(gnirndwi_label):
    os.makedirs(gnirndwi_label)

for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_gnirndwi')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(gnirndwi_label, new_name))

/tmp/ipykernel_911883/1353729320.py:17: RuntimeWarning: invalid value encountered in cast
  ndwi = np.round(ndwi).astype(np.uint8)


## Generate 'RGB + NIR' (4 bands) images & masks
- This cell generate 'RGB + NIR' (4 bands) images from the **images/color** and **images/ir** folders & saves them in **images/coir** folder
- This cell generates a label mask named **labels/mask_coir** for 'RGB + NIR' images. This label mask is similar to the **labels/mask_co** label mask, with the only difference being the name of each respective folder

In [7]:
# RGB + NIR _ 4 bands
rgb_dir = './images/color'
nir_dir = './images/ir'

coir_dir = './images/coir'
if not os.path.exists(coir_dir):
    os.makedirs(coir_dir)

# IMAGES
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    coir_path = os.path.join(coir_dir, name + '_coir.tiff')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    nir_band = cv2.imread(nir_path)[:, :, 0]    

    # Stack the arrays along the third axis (4 bands)
    combined_arr = np.stack([rgb_img[:, :, 0], rgb_img[:, :, 1], rgb_img[:, :, 2], nir_band], axis=2)

    # Save the new image as a .tiff file
    tiff.imwrite(coir_path, combined_arr)
    
# LABELS
co_label = './labels/mask_co'
gnirndwi_label = './labels/mask_coir'
if not os.path.exists(gnirndwi_label):
    os.makedirs(gnirndwi_label)

for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_coir')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(gnirndwi_label, new_name))

## Generate 'RGB + NDWI' (4 bands) images & masks
- This cell generate 'RGB + NDWI' (4 bands) images from the **images/color** and **images/ir** folders & saves them in **images/condwi** folder
- This cell generates a label mask named **labels/mask_condwi** for 'RGB + NDWI' images. This label mask is similar to the **labels/mask_co** label mask, with the only difference being the name of each respective folder

In [8]:
# RGB + NDWI _ 4 bands
rgb_dir = './images/color'
nir_dir = './images/ir'

condwi_dir = './images/condwi'
if not os.path.exists(condwi_dir):
    os.makedirs(condwi_dir)

# IMAGES
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    condwi_path = os.path.join(condwi_dir, name + '_condwi.tiff')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    nir = cv2.imread(nir_path)[:, :, 0]
    green = rgb_img[:, :, 1]
    ndwi = NDWI(green, nir)
    ndwi = np.round(ndwi).astype(np.uint8)

    # Stack the arrays along the third axis (4 bands)
    combined_arr = np.stack([rgb_img[:, :, 0], rgb_img[:, :, 1], rgb_img[:, :, 2], ndwi], axis=2)

    # Save the new image as a .tiff file
    tiff.imwrite(condwi_path, combined_arr)

# LABELS
co_label = './labels/mask_co'
gnirndwi_label = './labels/mask_condwi'
if not os.path.exists(gnirndwi_label):
    os.makedirs(gnirndwi_label)

for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_condwi')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(gnirndwi_label, new_name))

/tmp/ipykernel_911883/4039952846.py:24: RuntimeWarning: invalid value encountered in cast
  ndwi = np.round(ndwi).astype(np.uint8)


## Generate 'RGB + Green + NIR + NDWI' (6 bands) images & masks
- This cell generate 'RGB + Green + NIR + NDWI' (6 bands) images from the **images/color** and **images/ir** folders & saves them in **images/cognirndwi** folder
- This cell generates a label mask named **labels/mask_cognirndwi** for 'RGB + Green + NIR + NDWI' images. This label mask is similar to the **labels/mask_co** label mask, with the only difference being the name of each respective folder

In [9]:
# RGB + (Green, NIR, NDWI) _ 6 bands
rgb_dir = './images/color'
nir_dir = './images/ir'

cognirndwi_dir = './images/cognirndwi'
if not os.path.exists(cognirndwi_dir):
    os.makedirs(cognirndwi_dir)

# Get list of file names
file_names = [jpg_f.split('.')[0][:-3] for jpg_f in os.listdir(rgb_dir) if jpg_f.endswith('.jpg')]

for name in file_names:
    # Full path rgb, nir
    rgb_path = os.path.join(rgb_dir, name + '_co.jpg')
    nir_path = os.path.join(nir_dir, name + '_ir.jpg')
    cognirndwi_path = os.path.join(cognirndwi_dir, name + '_cognirndwi.tiff')
    
    # Read the images
    rgb_img = cv2.imread(rgb_path)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    
    # Construct combination (G, NIR, NDWI)
    green = rgb_img[:, :, 1]
    nir = cv2.imread(nir_path)[:, :, 0]
    gnirndwi_img = GreenNirNDWI(np.stack([green, nir], axis=2))

    # Stack the arrays along the third axis (4 bands)
    combined_arr = np.stack([rgb_img[:, :, 0], rgb_img[:, :, 1], rgb_img[:, :, 2],
                             gnirndwi_img[:, :, 0], gnirndwi_img[:, :, 1], gnirndwi_img[:, :, 2]], axis=2)

    # Save the new image as a .tiff file
    tiff.imwrite(cognirndwi_path, combined_arr)
    
# LABELS
co_label = './labels/mask_co'
gnirndwi_label = './labels/mask_cognirndwi'
if not os.path.exists(gnirndwi_label):
    os.makedirs(gnirndwi_label)

for source_name in os.listdir(co_label):
    if source_name.endswith('.png'):
        # Modify the file name as needed
        new_name = source_name.replace('_co', '_cognirndwi')
        shutil.copy(os.path.join(co_label, source_name), os.path.join(gnirndwi_label, new_name))

/tmp/ipykernel_911883/1353729320.py:17: RuntimeWarning: invalid value encountered in cast
  ndwi = np.round(ndwi).astype(np.uint8)


## Test

In [10]:
import tifffile as tiff

# Define the path to the .tiff image
image_path = './images/condwi/DJI_0020_103_condwi.tiff'

# Read the image
image = tiff.imread(image_path)
print(image.shape)

(1300, 1600, 4)
